<h2><b>Scraping data about books from the https://books.toscrape.com/ page</b></h2>

<h3>Libraries</h3>

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import seaborn as sns

<h3>The code:</h3>
<br>Iteration from page 1 to 50</br>
<br>Getting and appending all the links to a list containing dictionaries</br>
<br>Using these links to get various data e.g. the title, price, stock</br>
<br>Appending all the data to a list containing dictionaries and transforming it to a dataframe</br>
<br>Cleaning the data</br>

In [15]:
s = Service('C:\webdrivers\chromedriver.exe')
driver = webdriver.Chrome(service=s)
all_links = []
for page in range(1,51):
    url = f'https://books.toscrape.com/catalogue/page-{page}.html'
    driver.get(url)
    driver.implicitly_wait(3)
    book_links = driver.find_elements(By.XPATH,'.//article[@class="product_pod"]//h3//a')
    for book_link in book_links:
        book_link = book_link.get_attribute('href')
        all_links.append({"link":book_link})
    time.sleep(1)

In [17]:
len(all_links)

1000

In [19]:
book_data = []
for each_link in all_links:
    driver.get(each_link["link"])
    driver.implicitly_wait(5)
    book_link = each_link
    book_title = driver.find_element(By.XPATH,'//div[@class="col-sm-6 product_main"]//h1').text.strip()
    book_price = driver.find_element(By.XPATH,'(//div[@class="col-sm-6 product_main"]//p)[1]').text.strip()
    book_stock = driver.find_element(By.XPATH,'(//div[@class="col-sm-6 product_main"]//p)[2]').text.strip()
    book_code = driver.find_element(By.XPATH,'(//table[@class="table table-striped"]//tbody//tr//td)[1]').text.strip()
    product_type = driver.find_element(By.XPATH,'(//table[@class="table table-striped"]//tbody//tr//td)[2]').text.strip()
    book_nr_reviews = driver.find_element(By.XPATH,'(//table[@class="table table-striped"]//tbody//tr//td)[7]').text.strip()
    book_data.append({"link":book_link,
                      "title":book_title,
                      "price":book_price,
                      "stock":book_stock,
                      "product_type":product_type,
                      "nr_reviews":book_nr_reviews})
    

In [22]:
df = pd.DataFrame(book_data)

In [23]:
df.head()

,link,title,price,stock,product_type,nr_reviews
0,{'link': 'https://books.toscrape.com/catalogue...,A Light in the Attic,£51.77,In stock (22 available),Books,0
1,{'link': 'https://books.toscrape.com/catalogue...,Tipping the Velvet,£53.74,In stock (20 available),Books,0
2,{'link': 'https://books.toscrape.com/catalogue...,Soumission,£50.10,In stock (20 available),Books,0
3,{'link': 'https://books.toscrape.com/catalogue...,Sharp Objects,£47.82,In stock (20 available),Books,0
4,{'link': 'https://books.toscrape.com/catalogue...,Sapiens: A Brief History of Humankind,£54.23,In stock (20 available),Books,0


In [45]:
def fix_dict(x):
    x = x['link']
    return x

In [47]:
df['link'] = df['link'].apply(fix_dict)

In [51]:
def strip(x):
    x = x.strip()
    return x

In [52]:
df['link'] = df['link'].apply(strip)

In [57]:
df.shape

(1000, 6)

In [59]:
df.head()

,link,title,price,stock,product_type,nr_reviews
0,https://books.toscrape.com/catalogue/a-light-i...,A Light in the Attic,£51.77,In stock (22 available),Books,0
1,https://books.toscrape.com/catalogue/tipping-t...,Tipping the Velvet,£53.74,In stock (20 available),Books,0
2,https://books.toscrape.com/catalogue/soumissio...,Soumission,£50.10,In stock (20 available),Books,0
3,https://books.toscrape.com/catalogue/sharp-obj...,Sharp Objects,£47.82,In stock (20 available),Books,0
4,https://books.toscrape.com/catalogue/sapiens-a...,Sapiens: A Brief History of Humankind,£54.23,In stock (20 available),Books,0


In [71]:
df['price'] = df['price'].apply(lambda x: x.split('£')[1].strip())
df['price'] = df['price'].astype(np.number)

In [79]:
df['stock'] = df['stock'].apply(lambda x: x.split('(')[1].split(')')[0].strip())

In [80]:
df.head()

,link,title,price,stock,product_type,nr_reviews
0,https://books.toscrape.com/catalogue/a-light-i...,A Light in the Attic,51.77,22 available,Books,0
1,https://books.toscrape.com/catalogue/tipping-t...,Tipping the Velvet,53.74,20 available,Books,0
2,https://books.toscrape.com/catalogue/soumissio...,Soumission,50.10,20 available,Books,0
3,https://books.toscrape.com/catalogue/sharp-obj...,Sharp Objects,47.82,20 available,Books,0
4,https://books.toscrape.com/catalogue/sapiens-a...,Sapiens: A Brief History of Humankind,54.23,20 available,Books,0


In [102]:
df['stock'] = df['stock'].apply(lambda x: x.strip().split(' ')[0]) 

In [104]:
df['stock'] = df['stock'].astype(np.int16)

In [109]:
df.head(10)

,link,title,price,stock,product_type,nr_reviews,available
0,https://books.toscrape.com/catalogue/a-light-i...,A Light in the Attic,51.77,22,Books,0,Available
1,https://books.toscrape.com/catalogue/tipping-t...,Tipping the Velvet,53.74,20,Books,0,Available
2,https://books.toscrape.com/catalogue/soumissio...,Soumission,50.10,20,Books,0,Available
3,https://books.toscrape.com/catalogue/sharp-obj...,Sharp Objects,47.82,20,Books,0,Available
4,https://books.toscrape.com/catalogue/sapiens-a...,Sapiens: A Brief History of Humankind,54.23,20,Books,0,Available
5,https://books.toscrape.com/catalogue/the-requi...,The Requiem Red,22.65,19,Books,0,Available
6,https://books.toscrape.com/catalogue/the-dirty...,The Dirty Little Secrets of Getting Your Dream...,33.34,19,Books,0,Available
7,https://books.toscrape.com/catalogue/the-comin...,The Coming Woman: A Novel Based on the Life of...,17.93,19,Books,0,Available
8,https://books.toscrape.com/catalogue/the-boys-...,The Boys in the Boat: Nine Americans and Their...,22.60,19,Books,0,Available
9,https://books.toscrape.com/catalogue/the-black...,The Black Maria,52.15,19,Books,0,Available
